In [ ]:
from pathlib import Path

In [ ]:
import sys

sys.path.append(str(Path.home() / Path("Development") / "iSparrow"))

In [ ]:
from iSparrow import PreprocessorBase
from iSparrow import ModelBase
from iSparrow import SparrowRecording
from iSparrow import SpeciesPredictorBase
import iSparrow.utils as utils

In [ ]:
import os
import pandas as pd
import tests.set_up_sparrow_env as sp
from watchdog.events import LoggingEventHandler, FileSystemEventHandler
from watchdog.observers import Observer
import time
from datetime import datetime
from copy import deepcopy
import yaml
import threading

In [ ]:
# make a mock install of sparrow. will be invisible in the future
sp.install()

In [ ]:
class AnalysisEventHandler(FileSystemEventHandler):
    def __init__(self, callback: callable, wait_event):
        self.wait_event = wait_event
        self.callback = callback

    def on_created(self, event):
        if Path(event.src_path).is_file() and Path(event.src_path).suffix == ".wav":
            self.wait_event.wait()
            self.callback(event.src_path)

In [ ]:
class Watcher:

    def _load_model(self):
        self.preprocessor_module = utils.load_module(
            "ppm", self.model_dir / Path(self.model_name) / "preprocessor.py"
        )

        self.model_module = utils.load_module(
            "md", self.model_dir / Path(self.model_name) / "model.py"
        )

    def _write_config(self):

        with open(self.output / "config.yml", "w") as ymlfile:
            yaml.safe_dump(self.config, ymlfile)

    def __init__(
        self,
        indir: str,
        outdir: str,
        model_dir: str,
        model_name: str,
        watch_in_background: bool = False,
        preprocessor_config: dict = {},
        model_config: dict = {},
        recording_config: dict = {},
        species_predictor_config: dict = {},
    ):

        # set up data to use
        self.input = Path(indir)
        self.outdir = outdir
        self.output = Path(self.outdir) / Path(datetime.now().strftime("%y%m%d_%H%M%S"))
        self.output.mkdir(exist_ok=True, parents=True)

        self.model_dir = Path(model_dir)
        self.model_name = model_name
        self.proprocessor_module = None
        self.model_module = None
        self.model = None
        self.preprocessor = None
        self.watch_in_background = watch_in_background
        # set up model for analysis
        self._load_model()

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            model_path=self.model_dir / model_name, **model_config
        )

        # process config file
        self.config = {
            "Analysis": {
                "input": str(self.input),
                "output": str(self.output),
                "model_dir": str(self.model_dir),
                "watch_in_background": self.watch_in_background,
                "Preprocessor": deepcopy(preprocessor_config),
                "Model": deepcopy(model_config),
                "Recording": deepcopy(recording_config),
                "SpeciesPredictor": deepcopy(species_predictor_config),
            }
        }

        self.config["Analysis"]["Model"]["model_name"] = model_name

        self._write_config()

        # process species range predictor
        if all(name in recording_config for name in ["date", "lat", "lon"]) and all(
            recording_config[name] is not None for name in ["date", "lat", "lon"]
        ):

            model_name == "birdnet_default"

            # we can use the species predictor
            species_predictor = SpeciesPredictorBase(
                model_path=self.model_dir / model_name,
                **species_predictor_config,
            )

            recording_config["species_predictor"] = species_predictor

        # create recording object
        # species predictor is applied here once and then used for all the analysis calls that may follow
        self.recording = SparrowRecording(
            self.preprocessor, self.model, "", **recording_config
        )

        self.results = []

    def change_model(
        self,
        model_name,
        preprocessor_config: dict = {},
        model_config: dict = {},
    ):

        # raise RuntimeError("Not yet supported, only untested skeleton code exists")
        # print("setting analyzer thread to wait")
        # self.stop()  # reset event flag to false to indicate that the thread should wait

        self.model_name = model_name

        self._load_model()

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            model_path=self.model_dir / model_name, **model_config
        )

        self.recording.set_analyzer(self.model, self.preprocessor)

        # make new output, update config file and write new config file
        self.output = Path(self.outdir) / Path(datetime.now().strftime("%y%m%d_%H%M%S"))

        self.output.mkdir(parents=True, exist_ok=True)

        self.config["Analysis"]["Model"]["model_name"] = model_name

        self.config["Analysis"]["output"] = str(self.output)

        self._write_config()

        # print("setting analyzer flag to go")
        # self.go_on()

    def analyze(self, filename: str):

        self.recording.path = filename

        self.recording.analyze()

        self.results.extend(self.recording.detections)

        self.save_results(suffix=Path(filename).stem)

    def save_results(self, suffix=""):
        pd.DataFrame(self.results).to_csv(self.output / Path(f"results_{suffix}.csv"))

    def run(
        self,
    ):
        def task(e):
            e.wait()
            observer = Observer()
            event_handler = AnalysisEventHandler(self.analyze, self.wait_event)
            observer.schedule(event_handler, self.input, recursive=True)
            observer.start()

            try:
                while True:
                    time.sleep(1)
            except KeyboardInterrupt:
                observer.stop()
            except Exception:
                print("Something went wrong")
                observer.stop()
            observer.join()

        if self.watch_in_background:
            self.wait_event = threading.Event()
            self.wait_event.set()
            self.watcher_thread = threading.Thread(target=task, args=(self.wait_event,))
            self.watcher_thread.daemon = True
            self.watcher_thread.start()
        else:
            task()

    def stop(self):
        if self.watch_in_background and self.watcher_thread.is_alive():
            self.wait_event.clear()

    def go_on(self):
        if self.watch_in_background and self.watcher_thread.is_alive():
            self.wait_event.set()

    def check_analysis(self):
        for filename in self.input.iterdir():
            if (
                self.output / Path(f"results_{str(filename.stem)}.csv").is_file()
                is False
            ):
                self.analyze(filename)

In [ ]:
preprocessor_cfg = {
    "sample_rate": 48000,
    "overlap": 0.0,
    "sample_secs": 3.0,
    "resample_type": "kaiser_fast",
}

model_cfg = {
    "num_threads": 1,
    "sigmoid_sensitivity": 1.0,
    "species_list_file": None,
}

recording_cfg = {
    "date": datetime(year=2022, month=5, day=10),
    "lat": 35.4244,
    "lon": -120.7463,
    "species_presence_threshold": 0.03,
    "min_conf": 0.25,
    "species_predictor": None,
}

species_predictor_cfg = {
    "use_cache": True,
    "num_threads": 1,
}

runner = Watcher(
    Path.home() / "iSparrow_data",
    Path.home() / "iSparrow_output",
    Path.home() / "iSparrow/models",
    "birdnet_default",
    watch_in_background=True,
    preprocessor_config=preprocessor_cfg,
    model_config=model_cfg,
    recording_config=recording_cfg,
    species_predictor_config=species_predictor_cfg,
)

In [ ]:
runner.run()

In [ ]:
preprocessor_cfg = {
    "sample_rate": 48000,
    "overlap": 0.0,
    "sample_secs": 3.0,
    "resample_type": "kaiser_fast",
}

model_cfg = {
    "num_threads": 1,
    "sigmoid_sensitivity": 1.0,
    "default_model_path": Path.home() / "iSparrow/models/birdnet_default",
}

runner.change_model(
    "birdnet_custom", preprocessor_config=preprocessor_cfg, model_config=model_cfg
)